# Logistic Regression - Implementation from Scratch

## Overview
Binary classification using the **sigmoid function** and **gradient descent**.

### Key Formulas
- **Sigmoid:** $\sigma(z) = \frac{1}{1 + e^{-z}}$
- **Hypothesis:** $\hat{y} = \sigma(X\theta)$
- **Cost (Log Loss):** $J(\theta) = -\frac{1}{m}\sum[y\log(\hat{y}) + (1-y)\log(1-\hat{y})]$
- **Gradient:** $\nabla J = \frac{1}{m}X^T(\hat{y} - y)$
- **Update:** $\theta := \theta - \alpha \nabla J$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

np.random.seed(42)

## 1. Generate and Visualize Dataset

In [ ]:
# Generate binary classification dataset
X, y = make_classification(
    n_samples=300, n_features=2, n_informative=2,
    n_redundant=0, n_clusters_per_class=1, random_state=42
)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f'Training: {X_train.shape[0]} samples')
print(f'Test: {X_test.shape[0]} samples')
print(f'Class distribution (train): {np.bincount(y_train)}')

# Visualize
plt.figure(figsize=(8, 6))
plt.scatter(X_train_scaled[y_train == 0, 0], X_train_scaled[y_train == 0, 1],
            alpha=0.6, label='Class 0', marker='o')
plt.scatter(X_train_scaled[y_train == 1, 0], X_train_scaled[y_train == 1, 1],
            alpha=0.6, label='Class 1', marker='s')
plt.xlabel('Feature 1 (scaled)')
plt.ylabel('Feature 2 (scaled)')
plt.title('Binary Classification Dataset')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 2. Core Functions

In [ ]:
def add_intercept(X):
    """Add column of ones for intercept term."""
    return np.c_[np.ones(X.shape[0]), X]


def sigmoid(z):
    """
    Sigmoid activation function.
    
    sigma(z) = 1 / (1 + e^(-z))
    
    Clips z to avoid overflow in exp.
    """
    z = np.clip(z, -500, 500)
    return 1 / (1 + np.exp(-z))


# Visualize sigmoid
z_vals = np.linspace(-10, 10, 200)
plt.figure(figsize=(8, 4))
plt.plot(z_vals, sigmoid(z_vals), 'b-', linewidth=2)
plt.axhline(y=0.5, color='r', linestyle='--', alpha=0.5, label='Threshold = 0.5')
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.3)
plt.xlabel('z')
plt.ylabel('σ(z)')
plt.title('Sigmoid Function')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
def compute_cost(X, y, theta):
    """
    Binary cross-entropy (log loss) cost function.
    
    J(theta) = -(1/m) * sum[ y*log(h) + (1-y)*log(1-h) ]
    """
    m = len(y)
    h = sigmoid(X @ theta)
    # Clip to avoid log(0)
    h = np.clip(h, 1e-10, 1 - 1e-10)
    cost = -(1/m) * (y @ np.log(h) + (1 - y) @ np.log(1 - h))
    return cost


def compute_gradient(X, y, theta):
    """
    Compute gradient of cost function.
    
    gradient = (1/m) * X^T * (sigmoid(X*theta) - y)
    """
    m = len(y)
    h = sigmoid(X @ theta)
    gradient = (1/m) * X.T @ (h - y)
    return gradient


print('Core functions defined: sigmoid, compute_cost, compute_gradient')

## 3. Gradient Descent Implementation

In [ ]:
def gradient_descent(X, y, theta, learning_rate, n_iterations):
    """
    Optimize theta using gradient descent.
    
    Args:
        X: Feature matrix with intercept (m, n+1)
        y: Target labels (m,)
        theta: Initial parameters (n+1,)
        learning_rate: Step size alpha
        n_iterations: Number of iterations
    
    Returns:
        theta: Optimized parameters
        cost_history: Cost at each iteration
    """
    cost_history = []
    theta = theta.copy()
    
    for i in range(n_iterations):
        # Compute gradient
        gradient = compute_gradient(X, y, theta)
        
        # Update parameters
        theta -= learning_rate * gradient
        
        # Record cost
        cost = compute_cost(X, y, theta)
        cost_history.append(cost)
        
        if (i + 1) % 200 == 0:
            print(f'Iteration {i+1:4d}: Cost = {cost:.6f}')
    
    return theta, cost_history

## 4. Train the Model

In [ ]:
# Prepare data
X_train_b = add_intercept(X_train_scaled)
X_test_b = add_intercept(X_test_scaled)

# Initialize parameters to zeros
initial_theta = np.zeros(X_train_b.shape[1])

# Hyperparameters
learning_rate = 0.1
n_iterations = 1000

print(f'Training with lr={learning_rate}, iterations={n_iterations}\n')

# Train
theta_opt, cost_history = gradient_descent(
    X_train_b, y_train, initial_theta, learning_rate, n_iterations
)

print(f'\nOptimal parameters:')
print(f'  theta_0 (intercept): {theta_opt[0]:.4f}')
print(f'  theta_1: {theta_opt[1]:.4f}')
print(f'  theta_2: {theta_opt[2]:.4f}')
print(f'  Final cost: {cost_history[-1]:.6f}')

## 5. Visualize Cost Convergence

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(cost_history, 'b-', linewidth=2)
plt.xlabel('Iteration')
plt.ylabel('Cost J(theta)')
plt.title('Cost Function Convergence (Log Loss)')
plt.grid(True, alpha=0.3)
plt.show()
print(f'Cost: {cost_history[0]:.4f} -> {cost_history[-1]:.4f}')

## 6. Make Predictions and Evaluate

In [ ]:
def predict_proba(X, theta):
    """Predict class probabilities."""
    return sigmoid(X @ theta)

def predict(X, theta, threshold=0.5):
    """Predict class labels."""
    return (predict_proba(X, theta) >= threshold).astype(int)

# Predictions
y_train_pred = predict(X_train_b, theta_opt)
y_test_pred = predict(X_test_b, theta_opt)

# Metrics
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print('=' * 50)
print('MODEL PERFORMANCE')
print('=' * 50)
print(f'Training Accuracy: {train_acc:.4f} ({train_acc*100:.1f}%)')
print(f'Test Accuracy:     {test_acc:.4f} ({test_acc*100:.1f}%)')
print('\nClassification Report (Test Set):')
print(classification_report(y_test, y_test_pred))
print('Confusion Matrix (Test Set):')
print(confusion_matrix(y_test, y_test_pred))

## 7. Visualize Decision Boundary

In [ ]:
def plot_decision_boundary(X, y, theta, title='Decision Boundary'):
    """Plot data points and decision boundary."""
    plt.figure(figsize=(10, 7))
    
    # Create mesh grid
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.linspace(x1_min, x1_max, 200),
                           np.linspace(x2_min, x2_max, 200))
    
    # Predict on grid
    grid = np.c_[np.ones(xx1.ravel().shape[0]), xx1.ravel(), xx2.ravel()]
    probs = predict_proba(grid, theta).reshape(xx1.shape)
    
    # Plot probability contours
    plt.contourf(xx1, xx2, probs, levels=50, cmap='RdYlBu', alpha=0.4)
    plt.colorbar(label='P(class=1)')
    
    # Decision boundary (where P = 0.5)
    plt.contour(xx1, xx2, probs, levels=[0.5], colors='black', linewidths=2)
    
    # Data points
    plt.scatter(X[y == 0, 0], X[y == 0, 1], c='blue', label='Class 0',
                edgecolors='k', alpha=0.7)
    plt.scatter(X[y == 1, 0], X[y == 1, 1], c='red', label='Class 1',
                edgecolors='k', alpha=0.7)
    
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

plot_decision_boundary(X_train_scaled, y_train, theta_opt, 'Training Set - Decision Boundary')
plot_decision_boundary(X_test_scaled, y_test, theta_opt, 'Test Set - Decision Boundary')

## 8. Full Class Implementation

In [ ]:
class LogisticRegressionScratch:
    """
    Logistic Regression from scratch using gradient descent.
    """
    def __init__(self, learning_rate=0.1, n_iterations=1000):
        self.lr = learning_rate
        self.n_iter = n_iterations
        self.theta = None
        self.cost_history = []
    
    def _sigmoid(self, z):
        z = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z))
    
    def _add_intercept(self, X):
        return np.c_[np.ones(X.shape[0]), X]
    
    def fit(self, X, y):
        X_b = self._add_intercept(X)
        m, n = X_b.shape
        self.theta = np.zeros(n)
        self.cost_history = []
        
        for _ in range(self.n_iter):
            h = self._sigmoid(X_b @ self.theta)
            gradient = (1/m) * X_b.T @ (h - y)
            self.theta -= self.lr * gradient
            
            h_clipped = np.clip(h, 1e-10, 1 - 1e-10)
            cost = -(1/m) * (y @ np.log(h_clipped) + (1-y) @ np.log(1-h_clipped))
            self.cost_history.append(cost)
        
        return self
    
    def predict_proba(self, X):
        X_b = self._add_intercept(X)
        return self._sigmoid(X_b @ self.theta)
    
    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X) >= threshold).astype(int)
    
    def score(self, X, y):
        return np.mean(self.predict(X) == y)


# Use the class
model = LogisticRegressionScratch(learning_rate=0.1, n_iterations=1000)
model.fit(X_train_scaled, y_train)

print(f'Training Accuracy: {model.score(X_train_scaled, y_train):.4f}')
print(f'Test Accuracy:     {model.score(X_test_scaled, y_test):.4f}')

## 9. Comparison with Scikit-learn

In [ ]:
from sklearn.linear_model import LogisticRegression

# Sklearn model
sk_model = LogisticRegression(random_state=42)
sk_model.fit(X_train_scaled, y_train)

print('Parameter Comparison:')
print('=' * 50)
print(f'Our model  -> intercept: {model.theta[0]:.4f}, coefs: {model.theta[1:]}')
print(f'Sklearn    -> intercept: {sk_model.intercept_[0]:.4f}, coefs: {sk_model.coef_[0]}')

print(f'\nAccuracy Comparison (Test):')
print(f'Our model:  {model.score(X_test_scaled, y_test):.4f}')
print(f'Sklearn:    {sk_model.score(X_test_scaled, y_test):.4f}')

## 10. Threshold Tuning

The default threshold is 0.5, but it can be adjusted based on the problem.

In [ ]:
# Evaluate different thresholds
thresholds = np.arange(0.1, 1.0, 0.1)
results = []

for t in thresholds:
    preds = (predict_proba(X_test_b, theta_opt) >= t).astype(int)
    acc = accuracy_score(y_test, preds)
    cm = confusion_matrix(y_test, preds)
    tp = cm[1, 1] if cm.shape == (2, 2) else 0
    fp = cm[0, 1] if cm.shape == (2, 2) else 0
    fn = cm[1, 0] if cm.shape == (2, 2) else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    results.append((t, acc, precision, recall))
    print(f'Threshold {t:.1f}: Acc={acc:.3f}, Precision={precision:.3f}, Recall={recall:.3f}')

# Plot
fig, ax = plt.subplots(figsize=(10, 5))
ts = [r[0] for r in results]
ax.plot(ts, [r[1] for r in results], 'b-o', label='Accuracy')
ax.plot(ts, [r[2] for r in results], 'g-s', label='Precision')
ax.plot(ts, [r[3] for r in results], 'r-^', label='Recall')
ax.set_xlabel('Threshold')
ax.set_ylabel('Score')
ax.set_title('Metrics vs Decision Threshold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

## Key Takeaways

1. **Sigmoid function** maps linear output to probability [0, 1]
2. **Log loss** (binary cross-entropy) is the correct cost function, not MSE
3. **Gradient formula** looks like linear regression: $(1/m)X^T(\hat{y} - y)$, but $\hat{y} = \sigma(X\theta)$
4. **No closed-form solution** - must use gradient descent or similar optimizer
5. **Feature scaling** is important for convergence
6. **Decision boundary** is linear (hyperplane where $X\theta = 0$)
7. **Threshold tuning** lets you trade off precision vs recall
8. Logistic Regression is a strong **baseline** for any classification task